# TruLens 和 Ollama

https://github.com/truera/trulens/blob/main/trulens_eval/examples/expositional/models/ollama_quickstart.ipynb

In [1]:
%%time
%%capture

! pip install trulens_eval==0.33.0 litellm==1.11.1 langchain==0.0.351

CPU times: user 15.7 ms, sys: 607 µs, total: 16.3 ms
Wall time: 2.24 s


In [2]:
%%time

# Imports main tools:
from trulens_eval import Feedback
from trulens_eval import Tru
from trulens_eval import TruChain

tru = Tru()
tru.reset_database()


# Imports from langchain to build app. You may need to install langchain first
# with the following:
# ! pip install langchain>=0.0.170
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.
CPU times: user 6.39 s, sys: 571 ms, total: 6.96 s
Wall time: 6.62 s


In [3]:
from langchain.llms import Ollama

ollama = Ollama(base_url="http://monkey:11434", model="qwen2")
print(ollama("why is the sky blue"))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


The sky appears blue due to a process called Rayleigh scattering. Sunlight consists of many colors, each one having its own wavelength (length) and frequency.

When sunlight reaches Earth's atmosphere from the sun, it encounters tiny gas molecules like nitrogen and oxygen, as well as dust particles and other small bits in the air. Light is composed of photons that can be thought of as miniature energy packets. When these light packets encounter a gas molecule or another particle in the air, they bump into it.

Rayleigh scattering occurs when shorter wavelength (higher frequency) colors like blue and violet are scattered more than longer wavelengths (lower frequencies) like red, orange, and yellow. This is because shorter wavelengths interact with particles in the air over smaller distances compared to longer wavelengths. In other words, a photon of light can be deflected by bumping into these particles if they are roughly the same size as the wavelength of the light.

In our atmosphere

In [4]:
%%time

full_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template="Provide a helpful response with relevant background information for the following: {prompt}",
        input_variables=["prompt"],
    )
)

chat_prompt_template = ChatPromptTemplate.from_messages([full_prompt])

chain = LLMChain(llm=ollama, prompt=chat_prompt_template, verbose=True)

CPU times: user 145 µs, sys: 13 µs, total: 158 µs
Wall time: 160 µs


In [5]:
%%time

prompt_input = "What is a good name for a store that sells colorful socks?"

llm_response = chain(prompt_input)

display(llm_response)



> Entering new LLMChain chain...
Prompt after formatting:
Human: Provide a helpful response with relevant background information for the following: What is a good name for a store that sells colorful socks?


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



> Finished chain.


{'prompt': 'What is a good name for a store that sells colorful socks?',
 'text': 'When naming a store that specializes in selling colorful socks, it\'s essential to choose a name that not only captures the essence of vibrant and playful aesthetics but also resonates with your target audience. Here are some creative name suggestions along with explanations on their meaning:\n\n1. **Socktacular** - This name combines "sock" with "tacular," suggesting something exciting or exceptional, implying that each pair sold is top-quality, colorful, and fun.\n\n2. **ColorPop Socks** - This name highlights the primary feature of your store: colorful socks. The word \'pop\' implies a burst of color, which is perfect for attracting attention to items in this category.\n\n3. **VividStep** - Combining "vivid" (indicating brightness and vibrancy) with "step," this name suggests that each step you take wearing their socks can be an expression of personal style.\n\n4. **SockScape** - This name plays on th

CPU times: user 47 ms, sys: 16.6 ms, total: 63.6 ms
Wall time: 9.82 s


In [6]:
# Initialize LiteLLM-based feedback function collection class:
import litellm

from trulens_eval.feedback.provider import LiteLLM

litellm.set_verbose = False

ollama_provider = LiteLLM(
    model_engine="ollama/qwen2", 
    api_base="http://monkey:11434"
)

# Define a relevance function using LiteLLM
relevance = Feedback(
    ollama_provider.relevance_with_cot_reasons
).on_input_output()
# By default this will check relevance on the main app input and main app
# output.

✅ In relevance_with_cot_reasons, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In relevance_with_cot_reasons, input response will be set to __record__.main_output or `Select.RecordOutput` .


In [7]:
ollama_provider.relevance_with_cot_reasons(
    "What is a good name for a store that sells colorful socks?",
    "Great question! Naming a store that sells colorful socks can be a fun and creative process. Here are some suggestions to consider: SoleMates: This name plays on the idea of socks being your soul mate or partner in crime for the day. It is catchy and easy to remember, and it conveys the idea that the store offers a wide variety of sock styles and colors.",
)


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers



RuntimeError: Endpoint litellm request failed 4 time(s): 
	litellm.ServiceUnavailableError: OllamaException: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fc0d6498940>: Failed to establish a new connection: [Errno 111] Connection refused'))
	litellm.ServiceUnavailableError: OllamaException: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fc0d64a1060>: Failed to establish a new connection: [Errno 111] Connection refused'))
	litellm.ServiceUnavailableError: OllamaException: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fc017d6d540>: Failed to establish a new connection: [Errno 111] Connection refused'))
	litellm.